In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
import pandas as pd
import numpy as np
import torch as T
import torch
import math
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import import_ipynb
from nemoursModel import *
import random
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from pickle import dump,load
#from helper_functions import *
#from dataset import get_data_loader
import torchvision.utils as utils
import argparse
from torch.autograd import Variable
from argparse import ArgumentParser
import matplotlib.pyplot as plt
%matplotlib inline

save_path = "data/saved_models/nemours20.tar"

if not os.path.exists("data/saved_models"):
    os.makedirs("data/saved_models")

importing Jupyter notebook from nemoursModel.ipynb


In [2]:
ARG_PARSER = ArgumentParser()

ARG_PARSER.add_argument('--nfeatures', default=947, type=int)
ARG_PARSER.add_argument('--dfeatures', default=43, type=int)
ARG_PARSER.add_argument('--ehidden', default=74, type=int)


ARG_PARSER.add_argument('--num_epochs', default=100, type=int)
ARG_PARSER.add_argument('--seq_len', default=40, type=int)
ARG_PARSER.add_argument('--pred_len', default=30, type=int)
ARG_PARSER.add_argument('--batch_size', default=16, type=int)
ARG_PARSER.add_argument('--patience', default=20, type=int)
ARG_PARSER.add_argument('--e_lrn_rate', default=0.1, type=float)
ARG_PARSER.add_argument('--g_lrn_rate', default=0.1, type=float)
ARG_PARSER.add_argument('--d_lrn_rate', default=0.001, type=float)
ARG_PARSER.add_argument('--resume_training', default=False)
ARG_PARSER.add_argument('--train', default=False)
ARG_PARSER.add_argument('--eval', default=True)
ARG_PARSER.add_argument('--autoencoder', default=False)
ARG_PARSER.add_argument('--noise', default=False)
ARG_PARSER.add_argument('--clip_value', default=0.01, type=float)


_StoreTrueAction(option_strings=['--feature_matching'], dest='feature_matching', nargs=0, const=True, default=False, type=None, choices=None, help=None, metavar=None)

In [3]:
ARGS = ARG_PARSER.parse_args(args=[])
MAX_SEQ_LEN = ARGS.seq_len
BATCH_SIZE = ARGS.batch_size
EPSILON = 1e-40

In [ ]:
def run_test(args, model):
    model['e'].eval()
    model['g'].eval()
    model['d'].eval()
    
    RLoss=0
    TBatches=0
    oBmi=[]
    iBmi=[]
    oAge=[]
    oSex=[]
    
    output=[]
    with T.autograd.no_grad():
        files = 'test/testGan20.csv'
    
        maskFiles = 'test/testGanMask20.csv'
        
        dataset = CSVDataset(files, int(args.seq_len*500),1356100,args.seq_len)
        maskDataset = CSVDataset(maskFiles, int(args.seq_len*500),1356100, args.seq_len)

        loader = DataLoader(dataset,batch_size=1,num_workers=0, shuffle=False)
        maskLoader = DataLoader(maskDataset,batch_size=1,num_workers=0, shuffle=False)

        loss={}

        #for every batch
        for batch_idx, allData in enumerate(zip(loader, maskLoader)):
            data,mask=allData
            data=data.squeeze()
            mask=mask.squeeze()
            
            #values to be predicted
            y = data.clone().detach()
            sex=y[:,:,947]
            age=y[:,:,945]
            y=y[:,:,653]
            data=data[:,:,0:947]
            
            #yOrig=yOrig[:,:,653]



            #------------remove data from to be predicted timestamps------------------
            for i in range(data.shape[0]):
                j=40
                k=30
                
                data[i,j-k:j,:]=0
                y[i,0:j-k]=0
                age[i,0:j-k]=0
                sex[i,0:j-k]=0
                mask[i,0:j-k,:]=0



            #------------E training------------------
            h,(h_n,c_n) = model['e'](data)

            #------------G training------------------
            z=h_n[1].repeat(1,args.seq_len).view(h_n[1].shape[0],args.seq_len,-1)
            h_n=T.unsqueeze(h_n[1],0)
            c_n=T.unsqueeze(c_n[1],0)   
            output.extend(h_n)
            decodedOutput = model['g'](z,(h_n,c_n))
            
            #------------R Loss------------------
            criterion = nn.MSELoss()
            y=y.cpu()
            age=age.cpu()
            sex=sex.cpu()
            decodedOutput=decodedOutput.cpu()
            mask=mask.cpu()

            #------------MASk values-----------------
            y=y.reshape(-1)
            age=age.reshape(-1)
            sex=sex.reshape(-1)
            mask=mask[:,:,653]
            mask=mask.reshape(-1)
            y=np.multiply(y,mask)
            age=np.multiply(age,mask)
            sex=np.multiply(sex,mask)


            outputBMI=decodedOutput
            outputBMI=outputBMI.reshape(-1)
            outputBMI=np.multiply(outputBMI,mask)
            

            #------------Supervised Loss------------------
            
            loss = (outputBMI-y)
            loss =  (loss).pow(2)
            loss = loss.sum()
            loss = (loss)/(np.count_nonzero(mask))
            loss = T.sqrt(loss)
            RLoss=RLoss+loss
            outBmi, inBmi, outAge, outSex = plotBmi(outputBMI, y, age, sex)
            oBmi.extend(outBmi)
            iBmi.extend(inBmi)
            oAge.extend(outAge)
            oSex.extend(outSex)

        TBatches=TBatches+batch_idx+1
        
    RLoss = RLoss/TBatches
    #print("===================================")
    print("Val R Loss:",RLoss)
    return oBmi, iBmi, oAge, oSex
                
                
                

In [ ]:
def plotBmi(outBmi , inBmi, outAge, outSex):
    
    outBmi = outBmi.cpu().detach().numpy()
    inBmi = inBmi.cpu().detach().numpy()
    outAge = outAge.cpu().detach().numpy()
    outSex = outSex.cpu().detach().numpy()
    
    outBmi=outBmi[outBmi!=0]
    inBmi=inBmi[inBmi!=0]
    outAge=outAge[outAge!=0]
    outSex=outSex[outSex!=0]
    
    print(outBmi)
    print(inBmi)
    print(outAge)
    print(outSex)
    
    return outBmi,inBmi, outAge, outSex

In [5]:
def run_evalFull(args, model):
    model['e'].eval()
    model['g'].eval()
    model['d'].eval()
    
    RLoss=0
    TBatches=0
    oBmi=[]
    iBmi=[]
    oAge=[]
    oSex=[]
    
    output=[]
    with T.autograd.no_grad():
        files = 'val/testGan20.csv'
    
        maskFiles = 'val/testGanMask20.csv'
        
        dataset = CSVDataset(files, int(args.seq_len*500),1356100,args.seq_len)
        maskDataset = CSVDataset(maskFiles, int(args.seq_len*500),1356100, args.seq_len)

        loader = DataLoader(dataset,batch_size=1,num_workers=0, shuffle=False)
        maskLoader = DataLoader(maskDataset,batch_size=1,num_workers=0, shuffle=False)

        loss={}

        #for every batch
        for batch_idx, allData in enumerate(zip(loader, maskLoader)):
            #print('batch: {}'.format(batch_idx))
            data,mask=allData
            data=data.squeeze()
            mask=mask.squeeze()
            
            #values to be predicted
            y = data.clone().detach()
            #Take only BMI Values
            y=y[:,:,653]
            #data=data[:,:,0:947]
            
            #------------remove data from to be predicted timestamps------------------
            for i in range(data.shape[0]):
                j=40
                k=30
                
                data[i,j-k:j,:]=0
                y[i,0:j-k]=0
                age[i,0:j-k]=0
                mask[i,0:j-k,:]=0
        
            #------------E training------------------
            h,(h_n,c_n) = model['e'](data)

            #------------G training------------------
            z=h_n[1].repeat(1,args.seq_len).view(h_n[1].shape[0],args.seq_len,-1)
            h_n=T.unsqueeze(h_n[1],0)
            c_n=T.unsqueeze(c_n[1],0)   
            output.extend(h_n)
            decodedOutput = model['g'](z,(h_n,c_n))

            #------------R Loss------------------
            criterion = nn.MSELoss()
            y=y.cpu()
            age=age.cpu()
            sex=sex.cpu()
            decodedOutput=decodedOutput.cpu()
            mask=mask.cpu()

            #------------MASK Values------------------
            y=y.reshape(-1)
            #Mask only for BMI values
            mask=mask[:,:,653]
            mask=mask.reshape(-1)
            y=np.multiply(y,mask)
            outputBMI=decodedOutput
            outputBMI=outputBMI.reshape(-1)
            outputBMI=np.multiply(outputBMI,mask)

            #------------Supervised Loss------------------
            loss = (outputBMI-y)
            loss =  (loss).pow(2)
            loss = loss.sum()
            loss = (loss)/(np.count_nonzero(mask))
            loss = T.sqrt(loss)
            RLoss=RLoss+loss
        

        TBatches=TBatches+batch_idx+1
    RLoss = RLoss/TBatches
    #print("===================================")
    print("Val R Loss:",RLoss)
    return RLoss
                
                
                


In [8]:
%matplotlib inline
def batch_run(model,data,mask,args, optimizer, criterion,RLoss, DLoss, GLoss):
    
    loss={}
    freeze_d=False
    data=T.mul(data,mask)

    #------------E training------------------
    h,(h_n,c_n) = model['e'](data)

    #Take only BMI
    data=data[:,:,653]
    
    data=T.unsqueeze(data,2)
    #print("Input Data: ",data.shape)
    
    #------------G training Inputs------------------
    z=h_n[1].repeat(1,args.seq_len).view(h_n[1].shape[0],args.seq_len,-1)
    h_n=T.unsqueeze(h_n[1],0)
    c_n=T.unsqueeze(c_n[1],0) 

    #------------ G training------------------
    decodedOutput = model['g'](z,(h_n,c_n))
    #paramsG=list(model['g'].parameters())

    #------------Move to CUDA------------------
    data=data.cuda()
    mask=mask[:,:,653]
    mask=T.unsqueeze(mask,2)
    mask=mask.cpu()

    #------------MASK Genrated output------------------
    mask=mask.reshape(-1)
    y = data.clone().detach()
    y=y.cpu()
    y=y.reshape(-1)
    y=np.multiply(y,mask)
    
    output=decodedOutput.clone().detach()
    output=output.cpu()
    output=output.reshape(-1)
    decoded=output * mask


    #------------Prepare labels for Discriminator------------------
    y_real = get_cuda(T.ones(data.shape[0]))
    y_fake = get_cuda(T.zeros(data.shape[0]))


    #------------R Loss------------------
    y=y.cuda()
    decoded=decoded.cuda()
    loss['g']=criterion['g'](y,decoded)
    RLoss=RLoss+math.sqrt(loss['g'].item())

    #------------D training------------------
    if not args.autoencoder:
        #print("Auto")
        if args.noise:
            #print("Noise")
            #------------Random Noise G training------------------
            h_noise = T.empty([data.shape[0], 400]).uniform_() # random vector
            z_noise=h_noise.repeat(1,args.seq_len).view(h_noise.shape[0],args.seq_len,-1)
            h_noise=T.unsqueeze(h_noise,0)
            c_noise = T.empty([1, c_n.shape[1], c_n.shape[2]]).uniform_()
            noiseOutput = model['g'](z_noise,(h_noise,c_noise))
            nLabels = model['d'](noiseOutput.detach(),flag="fake")

            rLabels = model['d'](data,flag="original")
            fLabels = model['d'](decodedOutput,flag="fake")
            loss['d'] = criterion['d'](rLabels,fLabels,nLabels)
            
            noise_loss= -T.mean(nLabels)
        
        #if No Noise
        optimizer['d'].zero_grad()
        rLabels = model['d'](data,flag="original")
        fLabels = model['d'](decodedOutput,flag="fake")
        loss['d'] = criterion['d'](rLabels,fLabels)
        
        DLoss=DLoss+loss['d'].item()

        loss['d'].backward(retain_graph=True)
        optimizer['d'].step()
        

        #------------GD Loss------------------
        #nLabels = model['d'](noiseOutput,flag="fake")
        decodedOutput = model['g'](z,(h_n,c_n))
        fLabels = model['d'](decodedOutput,flag="fake")
        gen_loss= -T.mean(fLabels)
        #noise_loss= -T.mean(nLabels)

        loss['gd'] = gen_loss# + noise_loss
        GLoss=GLoss+loss['gd'].item()
        loss['g']=loss['g']+loss['gd']

    #------------E G training------------------
    optimizer['e'].zero_grad()
    optimizer['g'].zero_grad()
    loss['g'].backward()
    optimizer['e'].step()
    optimizer['g'].step()
    
    return RLoss, DLoss, GLoss

In [10]:
%matplotlib inline
def run_epoch(args, model, optimizer, criterion):
    ''' Run a single epoch
    '''
    
    decodedOutput=[]
    trainLoss=[]
    valLoss=[]
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=args.patience, verbose=True)
    if args.resume_training:
        early_stopping(133, model, optimizer, save_path)
    #for evrey epoch
    for epoch in range(args.num_epochs):
        model['e'].train()
        model['g'].train()
        model['d'].train()
    
        #Running Losses
        RLoss=0
        DLoss=0
        GLoss=0
        TBatches=0  
        print("=============EPOCH=================")
        freeze_d=False
        #if epoch<=args.g_pretraining_epochs:
            #freeze_d=True
       
        path = 'train/Data20/'
        files = list(map(lambda x : path + x, (filter(lambda x : x.endswith("csv"), os.listdir(path)))))
        
        maskPath = 'train/Data20/mask/'
        maskFiles = list(map(lambda x : maskPath + x, (filter(lambda x : x.endswith("csv"), os.listdir(maskPath)))))
        
        ids=[0,1,2,3]
        for i in ids:
            
            #print("====================New File========================")
            dataset = CSVDataset(files[i], int(args.seq_len*500),1356100,args.seq_len)
            maskDataset = CSVDataset(maskFiles[i], int(args.seq_len*500),1356100, args.seq_len)

            loader = DataLoader(dataset,batch_size=1,num_workers=0, shuffle=False)#number of times getitem is called in one iteration
            maskLoader = DataLoader(maskDataset,batch_size=1,num_workers=0, shuffle=False)


            #for every batch
            for batch_idx, allData in enumerate(zip(loader, maskLoader)):
                #print('batch: {}'.format(batch_idx))
                data,mask=allData

                data=data.squeeze()
                mask=mask.squeeze()

                RLoss, DLoss, GLoss = batch_run(model,data,mask,args, optimizer, criterion,RLoss, DLoss, GLoss)
                
                T.cuda.empty_cache()

            TBatches=TBatches+batch_idx+1
    
        RLoss=RLoss/TBatches
        DLoss=DLoss/TBatches
        GLoss=GLoss/TBatches   
        
        trainLoss.append(RLoss)

        valid_loss = run_evalFull(args, model)
        
        valLoss.append(valid_loss)

        print("epoch:", epoch, "loss_R:", "%.4f"%RLoss, "loss_G:", "%.4f"%GLoss, "loss_D:", "%.4f"%DLoss)


        if early_stopping.early_stop:
            print("Early stopping")
            break

    return trainLoss, valLoss


In [11]:
class DisLoss(nn.Module):
    ''' C-RNN-GAN discriminator loss
    '''
    def __init__(self):
        super(DisLoss, self).__init__()

    def forward(self, logits_real, logits_gen):#, logits_noise):
        ''' Discriminator loss
        logits_real: logits from D, when input is real
        logits_gen: logits from D, when input is from Generator
        '''
        d_loss_real = logits_real

        d_loss_gen = logits_gen# + logits_noise

        batch_loss = -(d_loss_real - d_loss_gen)
        return torch.mean(batch_loss)

In [12]:
def main(args):
    ''' Training sequence
    '''
    train_on_gpu = T.cuda.is_available()
    if train_on_gpu:
        print('Training on GPU.')
    else:
        print('No GPU available, training on CPU.')
        
    #Create Models
    model = {
        'e': Encoder(args.nfeatures, args.ehidden, use_cuda=train_on_gpu),
        'g': Generator(args.nfeatures, args.ehidden, use_cuda=train_on_gpu),
        'd': Discriminator(args.nfeatures, args.ehidden, use_cuda=train_on_gpu)
    }

    
    optimizer = {
        'e': optim.RMSprop(model['e'].parameters(), lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0.01, momentum=0.9, centered=False),
        'g': optim.RMSprop(model['g'].parameters(), lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0.01, momentum=0.9, centered=False),
        'd': optim.AdamW(model['d'].parameters(), args.d_lrn_rate)
        
    }
    criterion = {
        'g': nn.MSELoss(reduction='sum'),
        'd': DisLoss()
    }
    
    if torch.cuda.is_available():
        model['e'].cuda()
        model['g'].cuda()
        model['d'].cuda()
        
    if args.resume_training:
        checkpoint = T.load(save_path)
        model['e'].load_state_dict(checkpoint['E_model'])
        model['g'].load_state_dict(checkpoint['G_model'])
        model['d'].load_state_dict(checkpoint['D_model'])
        optimizer['e'].load_state_dict(checkpoint['E_trainer'])
        optimizer['g'].load_state_dict(checkpoint['G_trainer'])
        optimizer['d'].load_state_dict(checkpoint['D_trainer'])
        #Save Updated Model
        
        output = run_epoch(args, model, optimizer, criterion) 
        
        return model,output
    
    elif args.train:
        trainloss, valLoss = run_epoch(args, model, optimizer, criterion) 
        
        return trainloss, valLoss
        
    elif args.eval:
        #load Model
        checkpoint = T.load(save_path)
        model['e'].load_state_dict(checkpoint['E_model'])
        model['g'].load_state_dict(checkpoint['G_model'])
        model['d'].load_state_dict(checkpoint['D_model'])
        optimizer['e'].load_state_dict(checkpoint['E_trainer'])
        optimizer['g'].load_state_dict(checkpoint['G_trainer'])
        optimizer['d'].load_state_dict(checkpoint['D_trainer'])
        oBmi, iBmi, oAge, oSex = run_test(args, model)
        
        return oBmi, iBmi, oAge, oSex
    
    #'epoch': epoch + 1,
    
    
    
        

In [ ]:
#oBmi, iBmi, oAge, oSex = main(ARGS)
trainloss, valLoss = main(ARGS)